- [Optimal Online Adaptive Electric Vehicle Charging](http://netlab.caltech.edu/assets/publications/Guo-2017-OLP.pdf)
- [Pyomo - Optimization Modeling in Python](https://pyomo.readthedocs.io/en/stable/)
- [ TeX functions supported by KaTeX](https://katex.org/docs/supported.html)

# OLP: Online Linear Programming

<br>

<div style="border:1px solid blue; border-radius: 5px; background-color: #DDEDF9; color: black; padding: 0px">
    <div style="background-color: darkblue; color: white; padding: 10px; border-radius: 5px 5px 0 0">
        <strong>‼ Note:</strong>
    </div>
    <BR>
    >  The outcomes of the excel-model are the same as the pyomo-model !!! 
    <BR>
    >  Check: <strong>objective outcome</strong> and the <strong>Power to be supplied</strong> (Variables) 
    <BR>
    <BR>
</div>

### Pyomo-0

🚧Under construction 🚧 still to be changed to new KPI based on % utilization and % energy delivered

#### Model

Simple model for only 1 EV/SE to determine the ``kW`` for the next ``n`` periods (n=5).  
The following ``Objective function`` needs to be minimalized

##### Objective Function

$$
\begin{array}{l}
\text{min} \displaystyle\sum_{j=1}^{n} \omega_{j}  (1 - \frac{x_{j}}{ex_{j}}), & \omega_{j} = \frac{pt_{j}}{\displaystyle\sum_{j=1}^{n} pt_{j}} 
\end{array} 
$$

##### Decision Variables

The charging power at time slot $j$ are the decision variables, $x_{j}$ 

##### Constraints

The charging power to be used ($x_{j}$) in kW is subject to 4 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.


$$
\begin{array}{llllll}
\text{s.t.} & x_{j} \geq 0  &  \forall j = 1, \ldots, n\\
& x_{j} \leq ex_{j} & \forall j = 1, \ldots, n \\
& x_{j} \leq ev_{j} & \forall j = 1, \ldots, n\\
& x_{j} \leq se_{j} & \forall j = 1, \ldots, n\\ 
\end{array} 
$$



$$
\begin{array}{ll}
\text{j =  time slot}
\end{array} 
$$

This example relates to the ``pyomo_0`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_0``) 

<img src="./images/pyomo_o.png" width="1000">

#### Code & Results

In [5]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

from pyomo.environ import *

# row reserved for future use
n = 5  # number of time periods in the future

EX_MPO = 14.72  # enexis max Power output for each time period (constant)
EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period


# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------


model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)

# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
ex_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.ex = Param(model.J, initialize=ex_kw_dv, default=EX_MPO)

# EV max Power input for each time period
ev_kw_dv = {}  # deviations from the default value for ev i and time period j
ev_kw_dv[3] = 5.00  # kW
ev_kw_dv[4] = 3.68  # kW
ev_kw_dv[5] = 0.00  # kW
#
#
#
#
#
#
model.ev = Param(model.J, initialize=ev_kw_dv, default=EV_MPI)

# EVSE max Power output for each time period
se_kw_dv = {}  # deviations from the default value for evse i and time period j
se_kw_dv[1] = 5.00  # kW
#
#
model.se = Param(model.J, initialize=se_kw_dv, default=SE_MPO)

# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return sum(model.ex[j] - model.x[j] for j in model.J)


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power <= enexis max power output for period j?
def demand_constraint_rule(model, j):
    return model.x[j] <= model.ex[j]


# is ev/se power <= ev max power input for period j?
def ev_mpi_constraint_rule(model, j):
    return model.x[j] <= model.ev[j]


# is ev/se power <= evse max power output for period j?
def se_mpo_constraint_rule(model, j):
    return model.x[j] <= model.se[j]


# the next lines create one constraint for each member of the set model.J
model.EX_MPO_Constraint = Constraint(model.J, rule=demand_constraint_rule)
model.EV_MPI_Constraint = Constraint(model.J, rule=ev_mpi_constraint_rule)
model.SE_MPO_Constraint = Constraint(model.J, rule=se_mpo_constraint_rule)


# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

tgc.pprint()
print("\n")
print(f"Power to be supplied EVSE 1: {[tgc.x[x].value for x in range(1, 6)]}")


print("\nObjective Outcome = ", value(tgc.OBJ))

1 RangeSet Declarations
    J : Dimen=1, Size=5, Bounds=(1, 5)
        Key  : Finite : Members
        None :   True :   [1:5]

3 Param Declarations
    ev : Size=5, Index=J, Domain=Any, Default=7.36, Mutable=False
        Key : Value
          3 :   5.0
          4 :  3.68
          5 :   0.0
    ex : Size=5, Index=J, Domain=Any, Default=14.72, Mutable=False
        Key : Value
    se : Size=5, Index=J, Domain=Any, Default=7.36, Mutable=False
        Key : Value
          1 :   5.0

1 Var Declarations
    x : Size=5, Index=J
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   5.0 :  None : False : False : NonNegativeReals
          2 :     0 :  7.36 :  None : False : False : NonNegativeReals
          3 :     0 :   5.0 :  None : False : False : NonNegativeReals
          4 :     0 :  3.68 :  None : False : False : NonNegativeReals
          5 :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    OBJ : Size=1, Index=No

### Pyomo-1

🚧Under construction 🚧 code to be adapted to new KPI's

#### Model

Next, we extend the ``pyomo-0`` model by allowing multiple EV/SE charging points, (m = 3).  
This results in the following ``Objective function`` to be minimized.

##### Objective Function

$$
\begin{array}{l}
\text{min} \displaystyle\sum_{j=1}^{n} \omega_{j}  (1 - \frac{\sum_{i=1}^{m} x_{ij}}{ex_{j}}), & \omega_{j} = \frac{pt_{j}}{\sum_{j=1}^{n} pt_{j}} 
\end{array} 
$$

##### Decision Variables

The charging power for EV/SE $i$ at time slot $j$ are the decision variables, $x_{ij}$ 

##### Constraints

The charging power to be used ($ x_{ij} $) in kW is subject to 4 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.


$$
\begin{array}{rllll}
\text{s.t.} & x_{ij} \geq 0  &  \forall i = 1, \ldots, m & \forall j = 1, \ldots, n\\
\sum_{i=1}^{m} & x_{ij} \leq ex_{j} & \forall i = 1, \ldots, m & \forall j = 1, \ldots, n\\
& x_{ij} \leq ev_{ij} &               \forall i = 1, \ldots, m & \forall j = 1, \ldots, n\\
& x_{ij} \leq se_{ij} &               \forall i = 1, \ldots, m & \forall j = 1, \ldots, n\\ 
\end{array} 
$$



$$
\begin{array}{ll}
\text{i =  EV/SE charge,} & \text{m = 3}  \\
\text{j =  time slot,} & \text{n = 5}
\end{array}
$$

This example relates to the ``pyomo_1`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_1``) 

<img src="./images/pyomo_1.png" width="1000">

#### Code & Results

In [6]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

from pyomo.environ import *

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

EX_MPO = 14.72  # enexis max Power output for each time period (constant)
EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period


# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)

# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
ex_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.ex = Param(model.J, initialize=ex_kw_dv, default=EX_MPO)

# EV max Power input for each time period
ev_kw_dv = {}  # deviations from the default value for ev i and time period j
ev_kw_dv[1, 3] = 5.00  # kW
ev_kw_dv[1, 4] = 3.68  # kW
ev_kw_dv[1, 5] = 0.00  # kW
ev_kw_dv[2, 3] = 5.00  # kW
ev_kw_dv[2, 4] = 3.68  # kW
ev_kw_dv[2, 5] = 0.00  # kW
ev_kw_dv[3, 3] = 5.00  # kW
ev_kw_dv[3, 4] = 3.68  # kW
ev_kw_dv[3, 5] = 0.00  # kW
model.ev = Param(model.I, model.J, initialize=ev_kw_dv, default=EV_MPI)

# EVSE max Power output for each time period
se_kw_dv = {}  # deviations from the default value for evse i and time period j
se_kw_dv[1, 1] = 5.00  # kW
se_kw_dv[2, 1] = 5.00  # kW
se_kw_dv[3, 1] = 5.00  # kW
model.se = Param(model.I, model.J, initialize=se_kw_dv, default=SE_MPO)

# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return sum(model.ex[j] - sum(model.x[i, j] for i in model.I) for j in model.J)


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def demand_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.ex[j]


# is ev/se power[i] <= ev[i] max power input for period j?
def ev_mpi_constraint_rule(model, i, j):
    return model.x[i, j] <= model.ev[i, j]


# is ev/se power[i] <= evse[i] max power output for period j?
def se_mpo_constraint_rule(model, i, j):
    return model.x[i, j] <= model.se[i, j]


# the next line creates one constraint for each member of the set model.J
model.EX_MPO_Constraint = Constraint(model.J, rule=demand_constraint_rule)
model.EV_MPI_Constraint = Constraint(model.I, model.J, rule=ev_mpi_constraint_rule)
model.SE_MPO_Constraint = Constraint(model.I, model.J, rule=se_mpo_constraint_rule)


# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

tgc.pprint()
print("\n")
print(f"Power to be supplied EVSE 1: {[tgc.x[1, x].value for x in range(1, 6)]}")
print(f"Power to be supplied EVSE 2: {[tgc.x[2, x].value for x in range(1, 6)]}")
print(f"Power to be supplied EVSE 3: {[tgc.x[3, x].value for x in range(1, 6)]}")
print("\nObjective Outcome = ", value(tgc.OBJ))

5 Set Declarations
    EV_MPI_Constraint_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :   15 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5)}
    SE_MPO_Constraint_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :   15 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5)}
    ev_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :   15 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5)}
    se_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :   15 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2,

### Pyomo-2

🚧Under construction 🚧

code to be adapted for new KPI's including customer satisfaction

#### Model

The Enexis Grid Connection (ECG) utilization (%) is the highest priority, as modelled in ``pyomo-1``.
 
Next, the customer satisfaction is taken into account as second priority. This will be measured as % of ``desired charge`` $dc_{i}$ which has been delivered at the time the customer leaves. The ``final charge`` $fc_{i}$ in kWh can be calculated when we know for each EV the parking time ($pt_{i}$).

$$
fc_{i} = \displaystyle\sum_{j=1}^{n}  pt_{j} x_{ij} \\
$$

Please note that the model shouldn't be penalized for unrealistic customer demands. The customers ``desired charge`` $dc_{i}$ in (kWh) may never be higher than the ``realistc charge`` $rc_{i}$, being the maximum energy which can be deliverd at maximum charging power during the time of parking!  

The customer satisfaction will be expressed as: 

$$
\text{customer satisfaction}_{i} =  \frac{fc_{i}}{rc_{i}}
$$

##### Objective Function

Modifying the ``pyomo-1`` model results in the following ``Objective function`` to be minimized:

$$
\text{min}  \displaystyle\sum_{j=1}^{n} \left( e_{j}  -  \displaystyle\sum_{i=1}^{m} x_{ij} \right) + \displaystyle\sum_{i=1}^{m} \left(1 - \frac{1}{rc_{i}} \displaystyle\sum_{j=1}^{n}  pt_{j} x_{ij} \right)
$$

$$
\begin{array}{l}
\text{min} \displaystyle\sum_{j=1}^{n} \omega_{j}  (1 - \frac{\sum_{i=1}^{m} x_{ij}}{ex_{j}}), & \omega_{j} = \frac{pt_{j}}{\sum_{j=1}^{n} pt_{j}} 
\end{array} 
$$

##### Decision Variables

The charging power for EV/SE $i$ at time slot $j$ are the decision variables, $x_{ij}$ 

##### Constraints

The charging power to be used ($x_{ij}$) in kW is subject to 5 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.
1. the ``final charge`` $fc_{i}$ of EV $i$  should be less or equal to the ``realistic charge`` $rc_{i}$  
(which is less or equal to the ``desired charge`` $dc_{i}$)


$$
\begin{array}{rllll}
\text{s.t.} & x_{ij} \geq 0  &  \forall i = 1, \ldots, m & \forall j = 1, \ldots, n\\
\sum_{i=1}^{m} & x_{ij} \leq ex_{j} & \forall i = 1, \ldots, m & \forall j = 1, \ldots, n\\
& x_{ij} \leq ev_{ij} &               \forall i = 1, \ldots, m & \forall j = 1, \ldots, n\\
& x_{ij} \leq se_{ij} &               \forall i = 1, \ldots, m & \forall j = 1, \ldots, n\\
\sum_{j=1}^{n} & pt_{j} x_{ij} \leq rc_{i} & \forall i = 1, \ldots, m & \forall j = 1, \ldots, n\\
\end{array} 
$$



$$
\begin{array}{ll}
\text{i =  EV/SE charge,} & \text{m = 3}  \\
\text{j =  time slot,} & \text{n = 5}
\end{array}
$$

This example relates to the ``pyomo_1`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_2``) 

<img src="./images/pyomo_2.png" width="1000">

#### Code & Results

### Pyomo-3

🚧Under construction 🚧

To nice too have: include energy costs in weighting factor

#### Model

Pyomo-2 to be extended with the costs for energy by changing the weighting factor


##### Objective Function


##### Decision Variables


##### Constraints



#### Excel: results (sheet ``pyomo_3``) 

<img src="./images/pyomo_3.png" width="1000">

#### Code & Results